# Used Car Auction Sales Data Analysis and Machine Learning

<hr>

## Import Libraries

In [11]:
# IMPORT LIBRARIES
try:
    # PYSPARK
    from pyspark.sql import SparkSession
    from pyspark import SparkContext
    from pyspark.sql import SQLContext
    from pyspark.sql import DataFrame
    import pyspark.sql.types as tp
    import pyspark.sql.functions as F
    from pyspark.sql.functions import udf
    from pyspark.sql.types import DoubleType
    
    #Py Spark ML Libraries
    from pyspark.ml.regression import GBTRegressor
    from pyspark.ml.regression import LinearRegression
    from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
    from pyspark.ml import Pipeline
    from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
    from pyspark.ml.evaluation import RegressionEvaluator
    from pyspark.mllib.stat import Statistics
    from pyspark.ml.feature import MinMaxScaler
    
    # OTHER LIBRARIES
    import pandas as pd
    import numpy as np
    import glob
    from functools import reduce
    import seaborn as sns
    import matplotlib.pyplot as plt
    from urllib.request import urlopen
    import datetime
    from pathlib import Path
    import six
    from datetime import datetime
    
    print('[SUCCESS]')

    #CATCH ERROR IMPORTING A LIBRARY
except ImportError as ie:
    raise ImportError(f'[Error importing]: {ie}')

[SUCCESS]


## Initializing Spark session and cluster for work enviornment

In [3]:
'''
INITIALIZING SPARK SESSION
- NAME IS SET FOR SPARK SESSION WHEN RUNNING ON LOCAL HOST
'''
# IF RUNNING ON DOCKER (LOCAL) 
spark = SparkSession.builder.master('local').config("spark.executor.memory", "1g").config("spark.driver.memory", "2g").appName('UsedCar_Project').getOrCreate()

# IF RUNNING ON AMAZON BUCKER (UNCOMMENT AND COMMENT THE TOP
# spark = SparkSession.builder.master('spark://spark-master:7077').config("spark.executor.memory", "1g").config("spark.driver.memory", "2g").appName('UsedCar_Project').getOrCreate()
print(f'[SUCCESSFULLY RUNNING SPARK SESSION] -- {datetime.now()}')

[SUCCESSFULLY RUNNING SPARK SESSION] -- 2022-04-23 02:48:15.827201


# Load Data

<hr>

In [7]:
def load_data():
    '''
    car_prices.csv is uploaded to a google bucket for public use. Since file is too large to push to GitHub for others to use from repo
    this function will load the data from the google bucket.
    
    You can run this function each time and it will not download a new dataset each time since the first time you run it, it will download locally to your directory.
    --- unless you delete it each time
    
    Function will check to make sure file is in the directory
    - if it is, load it to a spark dataframe
    - if it is not, download it, then load it to a spark dataframe
    
    SCHEMA:
    - Created a schema to make sure the data types for the file being read is kept
    
    
    Drop Randome Values in state columns
    
    
    WARNING: TO USE THIS FUNCTION, YOU HAVE TO BE RUNNING JUPYTER NOTEBOOK ON A LINUX SERVER (USE DOCKER)
    
    NOTES:
    option("header",True).option("inferSchema", True)
    '''
    
    # CHECKS TO SEE IF FILE EXIST
    path = Path('car_prices.csv') 
    
    # IF FILE DOES NOT EXIST
    if not path.is_file():
        !wget https://storage.googleapis.com/iamangelsh-public-datasets/car_prices.csv 
    
    
    
    # CREATE SCHEMA TO KEEP DATA TYPES
    schema = tp.StructType([tp.StructField('year', tp.IntegerType(), True),
                           tp.StructField('make', tp.StringType(), True),
                           tp.StructField('model', tp.StringType(), True),
                           tp.StructField('trim', tp.StringType(), True),
                           tp.StructField('body', tp.StringType(), True),
                           tp.StructField('transmission', tp.StringType(), True),
                           tp.StructField('vin', tp.StringType(), True),
                           tp.StructField('state', tp.StringType(), True),
                           tp.StructField('condition', tp.DoubleType(), True),
                           tp.StructField('odometer', tp.DoubleType(), True),
                           tp.StructField('color', tp.StringType(), True),
                           tp.StructField('interior', tp.StringType(), True),
                           tp.StructField('seller', tp.StringType(), True),
                           tp.StructField('mmr', tp.IntegerType(), True),
                           tp.StructField('sellingprice', tp.IntegerType(), True),
                           tp.StructField('saledate', tp.StringType(), True)])
    
    
    # LOAD IN DATA WITH SCHEMA
    df = spark.read.csv("car_prices.csv", header = True, sep=",", schema=schema)
    
    old_df = df
    
    
    # FILTER OUT VIN NUMBERS FROM STATE COLUMN
    df = df.where(F.length(F.col("state")) <= 2)
    
    # DROP ROWS THAT CONTAIN NULL VALUES
    df = df.na.drop('any')
    
    # CREATE THRESHOLD FOR CONDITION COLUMN
    df = df.withColumn(
        'condition', 
        F.when(df.condition > 3.75, 'Great'
        ).when((df.condition >= 2) & (df.condition <= 3.75), 'Average'
        ).when(df.condition < 2, 'Bad'))
    
    # DROP COLUMNS THAT WON'T BE USED
    cols = ('trim', 'vin', 'interior', 'seller')
    df = df.drop(*cols)
    
    # USE MM DD YYYY FOR SALEDATE COLUMN
    df = df.withColumn(
        'saledate', F.substring('saledate', 5,11)
        ).withColumn(
        'saledate_year', F.substring('saledate', 7,5)
        ).withColumn(
        'saledate_month', F.substring('saledate', 1,3))
    
    df = df.withColumn(
        'saledate_year', F.col('saledate_year').cast(tp.IntegerType())
        )
    
    # RETURN NEW DATAFRAME
    return df, old_df


# LOAD THE DATA
df, old_df = load_data()

# SHOW DATA
df.show(5)

# SHOW NUMBER OF COLUMNS AND ROWS
print('Data for uncleaned DataFrame')
print(f'Number of columns: {len(old_df.columns)} \nNumber of Rows: {old_df.count()}')
print()
print('Data for cleaned DataFrame')
print(f'Number of columns: {len(df.columns)} \nNumber of Rows: {df.count()}')


# SHOW SCHEMA - DATATYPES
df.printSchema()

+----+-----+-------------------+-----+------------+-----+---------+--------+-----+-----+------------+-----------+-------------+--------------+
|year| make|              model| body|transmission|state|condition|odometer|color|  mmr|sellingprice|   saledate|saledate_year|saledate_month|
+----+-----+-------------------+-----+------------+-----+---------+--------+-----+-----+------------+-----------+-------------+--------------+
|2015|  Kia|            Sorento|  SUV|   automatic|   ca|    Great| 16639.0|white|20500|       21500|Dec 16 2014|         2014|           Dec|
|2015|  Kia|            Sorento|  SUV|   automatic|   ca|    Great|  9393.0|white|20800|       21500|Dec 16 2014|         2014|           Dec|
|2014|  BMW|           3 Series|Sedan|   automatic|   ca|    Great|  1331.0| gray|31900|       30000|Jan 15 2015|         2015|           Jan|
|2015|Volvo|                S60|Sedan|   automatic|   ca|    Great| 14282.0|white|27500|       27750|Jan 29 2015|         2015|           Jan|

# Preliminary Data Analysis

<hr>

## Null Values

In [8]:
def null_values(df):
    '''
    TAKES IN A DATAFRAME AND RETURNS THE COUNT OF NULL VALUES IN EACH COLUMN
    '''
    return df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])

NULL_VAL_COUNTS_OLD_DF = null_values(old_df)
NULL_VAL_COUNTS_OLD_DF.show()

print('After dropping Null values')
print()
NULL_VAL_COUNTS = null_values(df)
NULL_VAL_COUNTS.show()


+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|year| make|model| trim| body|transmission|vin|state|condition|odometer|color|interior|seller|mmr|sellingprice|saledate|
+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|   0|10301|10399|10651|13195|       65353|  4|    0|    11820|      94|  749|     749|     0| 26|           0|       0|
+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+

After dropping Null values

+----+----+-----+----+------------+-----+---------+--------+-----+---+------------+--------+-------------+--------------+
|year|make|model|body|transmission|state|condition|odometer|color|mmr|sellingprice|saledate|saledate_year|saledate_month|
+----+----+-----+----+------------+-----+---------+--------+-----+---+------------+--------+-------------+--------------+


## Look for Outliers

In [9]:
# CREATE A TEMPORARY VIEW TO QUERY
df.createOrReplaceTempView("usedCars")

# LOOK FOR OUTLIERS
spark.sql('select * from usedCars where sellingPrice = 1').show() # two records where sellingPrice = 1: these should be removed

spark.sql('select * from usedCars where odometer = 999999.0').show()
spark.sql('select count(*) as records from usedCars where odometer = 999999.0').show() # sixty records where odometer = 999,999.0; these should be removed

spark.sql('select * from usedCars where mmr < 100 order by mmr').show() # mmr values are significantly lower than sale date, but looks reasonable

spark.sql('select avg(mmr) as avg_mmr, avg(sellingprice) as avg_sellprice from usedCars').show() # mmr is about $140 higher than sale price



+----+-------------+------------+------------+------------+-----+---------+--------+-----+-----+------------+-----------+-------------+--------------+
|year|         make|       model|        body|transmission|state|condition|odometer|color|  mmr|sellingprice|   saledate|saledate_year|saledate_month|
+----+-------------+------------+------------+------------+-----+---------+--------+-----+-----+------------+-----------+-------------+--------------+
|2003|Mercedes-Benz|     E-Class|       Sedan|   automatic|   ga|  Average|     1.0|black| 7325|           1|Jan 06 2015|         2015|           Jan|
|2014|         Ford|E-Series Van|E-Series Van|   automatic|   tx|    Great| 31886.0|white|20800|           1|Feb 12 2015|         2015|           Feb|
+----+-------------+------------+------------+------------+-----+---------+--------+-----+-----+------------+-----------+-------------+--------------+

+----+---------+--------------+-------------+------------+-----+---------+--------+------+---

In [10]:
def filter_outliers(df):
    '''
    FUNCTION TAKES IN 1 PARAMETER
    
    PARAM 1 --> Dataframe to adjust
    
    Will return a new dataframe without the outliers
    '''
    
    # FILTER OUT SELLINGPRICE = 1 AND ODOMETER = 999999.0
    df = df.where(F.col("sellingprice") != 1)
    df = df.where(F.col("odometer") != 999999.0)
    
    return df

df = filter_outliers(df)

## Scale Continous Variables

In [12]:
def scale_cont_var(df):
    # UDF for converting column type from vector to double type
    unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

    # Iterating over columns to be scaled
    for i in ["year", "odometer", "sellingprice"]:
        # VectorAssembler Transformation - Converting column to vector type
        assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

        # MinMaxScaler Transformation
        scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

        # Pipeline of VectorAssembler and MinMaxScaler
        pipeline = Pipeline(stages=[assembler, scaler])

        # Fitting pipeline on dataframe
        df = pipeline.fit(df).transform(df).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")

    print("Scaled Dataframe:")
    df.show(5)
    
scale_cont_var(df)

Scaled Dataframe:
+----+-----+-------------------+-----+------------+-----+---------+--------+-----+-----+------------+-----------+-------------+--------------+-----------+---------------+-------------------+
|year| make|              model| body|transmission|state|condition|odometer|color|  mmr|sellingprice|   saledate|saledate_year|saledate_month|year_Scaled|odometer_Scaled|sellingprice_Scaled|
+----+-----+-------------------+-----+------------+-----+---------+--------+-----+-----+------------+-----------+-------------+--------------+-----------+---------------+-------------------+
|2015|  Kia|            Sorento|  SUV|   automatic|   ca|    Great| 16639.0|white|20500|       21500|Dec 16 2014|         2014|           Dec|        1.0|          0.017|              0.093|
|2015|  Kia|            Sorento|  SUV|   automatic|   ca|    Great|  9393.0|white|20800|       21500|Dec 16 2014|         2014|           Dec|        1.0|           0.01|              0.093|
|2014|  BMW|           3 Se

## Look for Clusters

In [13]:
def sql_query_clusters(df):
    # CREATE A TEMPORARY VIEW TO QUERY
    df.createOrReplaceTempView("usedCars")

    spark.sql('select make, avg(sellingprice) as avg_price from usedCars group by make order by avg_price desc').show()

    spark.sql('select make, model, avg(sellingprice) as avg_price from usedCars group by make, model order by avg_price desc').show() # make-model looks promising, maybe quartiles?
    
sql_query_clusters(df)

+-------------+------------------+
|         make|         avg_price|
+-------------+------------------+
|  Rolls-Royce|         153456.25|
|      Ferrari|128852.94117647059|
|  Lamborghini|          111500.0|
|      Bentley| 72713.33333333333|
|        Tesla| 67054.34782608696|
| Aston Martin|           55500.0|
|       Fisker| 46461.11111111111|
|     Maserati| 43729.81651376147|
|        Lotus|           40800.0|
|      Porsche|38932.109766637856|
|   Land Rover| 33225.28744326778|
|          Ram|25257.458209693374|
|Mercedes-Benz|21320.646172522138|
|          BMW|21293.283902661944|
|     Infiniti| 20562.79630290486|
|        Lexus|20286.116929285166|
|         Audi|20010.167638483967|
|       Jaguar|19429.853619729514|
|      Lincoln| 17547.19143321153|
|          GMC| 16769.46783118151|
+-------------+------------------+
only showing top 20 rows

+-------------+--------------------+------------------+
|         make|               model|         avg_price|
+-------------+-------

# Exploratory Data Analysis

<hr>


PUT IN WORK FOR 
JERRY, PRAFULLA, SARAH

# Modeling and Predictions
<hr>


In [48]:
def data_processing(df):
    '''
    FUNCTION TAKES IN 2 PARAMETERS:
    1. Data Frame --> data frame that you are working with that you want to process
    2. Column Names --> these are the categorical columns that will be used for processing/transformed
    
    Methods applied:
    1. Indexing --> Get index of string columns
    2. One hot encoding --> categorical values to numerical values
    3. Assembler --> vectorizing encoded values
    4. Pipeline --> create a pipeline do bring all these processes together
    
    Returns a transformed model as a dataframe
    '''
    
    # 1. INDEXER
    
    cc = names = ['year', 'make', 'model', 'body', 'transmission', 'state', 'condition',
       'odometer', 'color', 'mmr', 'saledate', 'saledate_year',
       'saledate_month', 'year_Scaled', 'odometer_Scaled',
       'sellingprice_Scaled']
    indexers = [StringIndexer(inputCol = column, outputCol = f'{column}_indexed') for column in cc]
    
    # 2. One Hot Encoding
    
    encoders = [OneHotEncoder(dropLast = False, inputCol = idx.getOutputCol(), outputCol = f'{idx.getOutputCol()}_encoded') for idx in indexers]
    
    # 3. Assembler --> Vectorize encoded values
    
    assembler = VectorAssembler(inputCols = [encoded_val.getOutputCol() for encoded_val in encoders], outputCol = 'features')
    
    # 4. Pipeline
    
    pipeline = Pipeline(stages = indexers + encoders + [assembler])
    
    
    # Return our transformed moder
    model = pipeline.fit(df)
    
    transformed_df = model.transform(df)
    
    return transformed_df


trans_df = data_processing(df)
# trans_df.printSchema()
trans_df = trans_df.select(['features', 'sellingprice'])

In [49]:
def train_test_split(df, split):
    '''
    FUNCTION TAKES IN 2 PARAMETERS:
    1. DATAFRAME --> Dataframe that will be split (the one from above that was transformed)
    2. SPLITS --> list with split1 and split 2 (ex: 0.7, 0.3)
    
    RETURN:
    - train_df : split[0]
    - test_df : split[1]
    '''
    
    splits = df.randomSplit(split)
    
    return splits[0], splits[1]


split_list = [0.7, 0.3]
df_train, df_test = train_test_split(trans_df, split_list)

In [50]:
def create_linear_reg_model(train, test):
    '''
    FUNCTION TAKES IN 2 PARAMETERS AND RETURNS A LINEAR MODEL
    
    PARAM 1 --> TRAIN dataset
    PARAM 2 --> TEST dataset
    
    
    OVERVIEW:
    Function will create a linear regression model and print RMSE and r2.
    It will aslo return the linear regression model for predictions
    '''
    
    lr_model = LinearRegression(
        featuresCol = 'features',
        labelCol = 'sellingprice',
        maxIter = 10,
        regParam = 0.3,
        elasticNetParam = 0.8
    )
    
    linear_model = lr_model.fit(train)
    
    trainSummary = linear_model.summary
    print(
        f'RMSE: {round(trainSummary.rootMeanSquaredError, 2)}'
    )
    print(
        f'r2: {round(trainSummary.r2, 2)}'
    )
    
    return linear_model
    
linear_model = create_linear_reg_model(df_train, df_test)

RMSE: 820.81
r2: 0.99


In [58]:
def create_grad_boost_tree_model(train, test):
    '''
    FUNCTION TAKES IN 2 PARAMETERS AND RETURNS A GRADIENT BOOSTED TREE REGRESSION MODEL
    
    PARAM 1 --> TRAIN dataset
    PARAM 2 --> TEST dataset
    
    
    OVERVIEW:
    Function will create a linear regression model and print RMSE and r2.
    It will aslo return the linear regression model for predictions
    '''
    
    gbt_model = GBTRegressor(
        fearturesCol = 'features',
        labelCol = 'sellingprice',
        maxIter = 10
    )
    
    
    gradientBoost_model = gbt_model.fit(train)
    
    trainSummary = gradientBoost_model.summary
    print(
        f'RMSE: {round(trainSummary.rootMeanSquaredError, 2)}'
    )
    print(
        f'r2: {round(trainSummary.r2, 2)}'
    )
    return gradientBoost_model


gbtr_model = create_linear_reg_model(df_train, df_test)

RMSE: 820.81
r2: 0.99


In [64]:
def make_predictions(model, test, model_name):
    '''
    FUNCTION TAKES IN 2 PARAMETERS
    
    PARAM 1 -- Model created
    PARAM 2 -- test dataset
    
    OVERVIEW:
    Function will create predictions and will evaluate the predictions for R2 and RMSE
    
    Will return a predictions dataframe
    '''
    
    predictions = model.transform(test)
    
    model_eval = RegressionEvaluator(
        predictionCol = 'prediction',
        labelCol = 'sellingprice',
        metricName = 'r2')
    modelEval = model.evaluate(test)
    
    
    print(f'Stats for {model_name}:')
    print(f'R2 on test data: {model_eval.evaluate(predictions)}')
    print(f'RMSE on test data: {modelEval.rootMeanSquaredError}')

    
    return predictions


In [65]:
linear_model_predictions = make_predictions(linear_model, df_test, 'Linear Model')

Stats for Linear Model:
R2 on test data: 0.9681359645408623
RMSE on test data: 1720.2704335014698


In [53]:
# Look at the predictions
linear_model_predictions.select('prediction', 'sellingprice').show()

+------------------+------------+
|        prediction|sellingprice|
+------------------+------------+
|13022.313186622936|       13600|
|15654.466222819825|       15900|
|16264.766153311339|       18000|
|14828.590856072722|       14000|
|16309.797190502122|       16200|
|10807.857743528712|       12400|
|13279.314760497566|       11700|
|11650.370012603827|        9800|
|7315.5448677606455|       10000|
| 11677.59749694638|       12700|
|10233.827069652802|       11700|
| 15506.25340481593|       16300|
|14477.345019574652|       15200|
|14432.875798095381|       15100|
|17194.175712102828|       18100|
|13811.492151328634|       13800|
|15456.514246224688|       15600|
| 14405.69604717595|       14300|
|16345.451954548962|       15700|
|17544.037236401164|       18700|
+------------------+------------+
only showing top 20 rows



In [59]:
gbtr_model_predictions = make_predictions(gbtr_model, df_test, 'Gradient Boosted Tree Regressor')

Stats for Gradient Boosted Tree Regressor:
R2 on test data: 0.9681359645408623
RMSE on test data: 1720.2704335014698


In [60]:
gbtr_model_predictions.select('prediction', 'sellingprice').show()

+------------------+------------+
|        prediction|sellingprice|
+------------------+------------+
|13022.313186622936|       13600|
|15654.466222819825|       15900|
|16264.766153311339|       18000|
|14828.590856072722|       14000|
|16309.797190502122|       16200|
|10807.857743528712|       12400|
|13279.314760497566|       11700|
|11650.370012603827|        9800|
|7315.5448677606455|       10000|
| 11677.59749694638|       12700|
|10233.827069652802|       11700|
| 15506.25340481593|       16300|
|14477.345019574652|       15200|
|14432.875798095381|       15100|
|17194.175712102828|       18100|
|13811.492151328634|       13800|
|15456.514246224688|       15600|
| 14405.69604717595|       14300|
|16345.451954548962|       15700|
|17544.037236401164|       18700|
+------------------+------------+
only showing top 20 rows

